In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="1RJjttCvltRK-j5XaI_Tp752cibGKRYMf", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/02_00_intro.mp3"))


In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

In [ ]:
#@title 🎧 Listen: Why Reverse Matters
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_02_why_reverse_matters.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Building Intuition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_03_building_intuition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


# The Reverse Process and DDPM Loss — Vizuara

## 1. Why Does This Matter?

In the previous notebook, we learned how to systematically destroy images with the forward diffusion process. Now comes the exciting part — learning to **reverse** this process. If we can undo each tiny step of noise addition, we can start from pure random noise and generate entirely new images.

The DDPM paper by Ho et al. (2020) showed that this reverse process can be trained with a surprisingly simple loss function: just predict the noise that was added. This single insight transformed diffusion models from a theoretical curiosity into a practical powerhouse.

By the end of this notebook, you will:
- Understand why the reverse posterior is tractable when conditioned on $x_0$
- Derive the DDPM loss from the variational lower bound
- Implement the noise prediction loss
- Train a simple denoiser and verify it learns to predict noise
- See the connection between noise prediction and score matching

## 2. Building Intuition

Let us think about what it means to denoise an image. Imagine you have a photograph of a cat with some static noise overlay. A human can easily recognize the cat and mentally "remove" the noise. But how would a neural network do it?

The key insight is this: if we know exactly what noise was added, we can subtract it to recover the clean image. So instead of learning to directly produce the clean image, we train the network to predict the noise.

Let us set up our environment and build on the forward process from Notebook 1.

In [ ]:
!pip install torch torchvision matplotlib numpy -q

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Recreate the forward diffusion from Notebook 1
T = 1000
betas = torch.linspace(1e-4, 0.02, T)
alphas = 1.0 - betas
alpha_bars = torch.cumprod(alphas, dim=0)

def forward_diffusion(x_0, t, noise=None):
    """Add noise to x_0 at timestep t."""
    if noise is None:
        noise = torch.randn_like(x_0)
    sqrt_ab = torch.sqrt(alpha_bars[t]).view(-1, 1, 1, 1)
    sqrt_1_ab = torch.sqrt(1 - alpha_bars[t]).view(-1, 1, 1, 1)
    x_t = sqrt_ab * x_0 + sqrt_1_ab * noise
    return x_t, noise

# Load MNIST
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

In [ ]:
#@title 🎧 Listen: True Posterior Math
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_04_true_posterior_math.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Ddpm Loss Math
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_05_ddpm_loss_math.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 3. The Mathematics

### The True Posterior

We want to learn $p(x_{t-1} | x_t)$ — given a noisy image, produce a slightly less noisy version. Computing this directly is intractable. But if we also know the original image $x_0$, the posterior becomes a simple Gaussian:

$$q(x_{t-1} | x_t, x_0) = \mathcal{N}(x_{t-1}; \tilde{\mu}_t, \tilde{\beta}_t I)$$

where:

$$\tilde{\mu}_t = \frac{\sqrt{\bar{\alpha}_{t-1}} \beta_t}{1 - \bar{\alpha}_t} x_0 + \frac{\sqrt{\alpha_t}(1 - \bar{\alpha}_{t-1})}{1 - \bar{\alpha}_t} x_t$$

Let us plug in numbers. At $t=500$: $\alpha_t = 0.99$, $\bar{\alpha}_t = 0.5$, $\bar{\alpha}_{t-1} = 0.505$, $\beta_t = 0.01$.
- Coefficient of $x_0$: $\frac{\sqrt{0.505} \times 0.01}{0.5} = \frac{0.711 \times 0.01}{0.5} = 0.0142$
- Coefficient of $x_t$: $\frac{\sqrt{0.99} \times 0.495}{0.5} = \frac{0.995 \times 0.495}{0.5} = 0.985$

So $\tilde{\mu}_t = 0.0142 \cdot x_0 + 0.985 \cdot x_t$. The posterior mean mostly relies on $x_t$ — which makes sense because $x_t$ already contains most of the useful information at intermediate noise levels.

### From Means to Noise Prediction

Since we cannot access $x_0$ during generation, we reparameterize. Using $x_0 = \frac{1}{\sqrt{\bar{\alpha}_t}}(x_t - \sqrt{1-\bar{\alpha}_t}\epsilon)$, the loss simplifies to:

$$L_{\text{simple}} = \mathbb{E}_{t, x_0, \epsilon}\left[\|\epsilon - \epsilon_\theta(x_t, t)\|^2\right]$$

This is the DDPM loss. It says: predict the noise that was added, and minimize the squared error. That is it!

Let us verify with numbers. If true noise $\epsilon = 0.3$ and predicted $\epsilon_\theta = 0.25$:
- Loss = $(0.3 - 0.25)^2 = 0.0025$
- Perfect prediction: loss = 0

## 4. Let's Build It — Component by Component

### Step 1: Compute the True Posterior

In [ ]:
#@title 🎧 Code Walkthrough: Code Posterior Mean
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_06_code_posterior_mean.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def compute_posterior_mean(x_0, x_t, t):
    """
    Compute the true posterior mean mu_tilde.

    mu_tilde = (sqrt(alpha_bar_{t-1}) * beta_t) / (1 - alpha_bar_t) * x_0
             + (sqrt(alpha_t) * (1 - alpha_bar_{t-1})) / (1 - alpha_bar_t) * x_t
    """
    alpha_bar_t = alpha_bars[t].view(-1, 1, 1, 1)
    alpha_bar_t_prev = alpha_bars[t - 1].view(-1, 1, 1, 1) if t.min() > 0 else torch.ones_like(alpha_bar_t)
    alpha_t = alphas[t].view(-1, 1, 1, 1)
    beta_t = betas[t].view(-1, 1, 1, 1)

    coeff_x0 = torch.sqrt(alpha_bar_t_prev) * beta_t / (1 - alpha_bar_t)
    coeff_xt = torch.sqrt(alpha_t) * (1 - alpha_bar_t_prev) / (1 - alpha_bar_t)

    return coeff_x0 * x_0 + coeff_xt * x_t

In [ ]:
#@title 🎧 What to Look For: Visualize Posterior
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_07_visualize_posterior.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### Step 2: Visualize the True Posterior in Action

In [ ]:
# Show that if we KNOW x_0, we can compute the reverse step
sample = dataset[0][0].unsqueeze(0)  # [1, 1, 28, 28]

# Go to t=500
t_val = torch.tensor([500])
x_500, noise = forward_diffusion(sample, t_val)

# Compute posterior mean (using known x_0)
mu_tilde = compute_posterior_mean(sample, x_500, t_val)

fig, axes = plt.subplots(1, 4, figsize=(16, 4))
axes[0].imshow(sample.squeeze().numpy(), cmap='gray')
axes[0].set_title('Original x₀')
axes[1].imshow(x_500.squeeze().clamp(-1, 1).detach().numpy(), cmap='gray')
axes[1].set_title('Noisy x₅₀₀')
axes[2].imshow(mu_tilde.squeeze().clamp(-1, 1).detach().numpy(), cmap='gray')
axes[2].set_title('Posterior Mean μ̃₅₀₀\n(using known x₀)')
axes[3].imshow(noise.squeeze().numpy(), cmap='gray')
axes[3].set_title('True Noise ε')
for ax in axes:
    ax.axis('off')
plt.suptitle('True Posterior: One Step of Denoising (with known x₀)', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Code Walkthrough: Simple Noise Predictor
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_08_simple_noise_predictor.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### Step 3: Build a Simple Noise Predictor

Before using a full U-Net, let us build a minimal CNN that can predict noise.

In [ ]:
class SimpleNoisePredictor(nn.Module):
    """
    A simple CNN that predicts noise given a noisy image and timestep.
    This is NOT a U-Net — just a minimal model to verify the loss works.
    """
    def __init__(self, T=1000, hidden_dim=64):
        super().__init__()
        # Timestep embedding
        self.time_embed = nn.Embedding(T, hidden_dim)

        # Encoder
        self.conv1 = nn.Conv2d(1, hidden_dim, 3, padding=1)
        self.conv2 = nn.Conv2d(hidden_dim, hidden_dim, 3, padding=1)
        self.conv3 = nn.Conv2d(hidden_dim, hidden_dim, 3, padding=1)

        # Decoder
        self.conv4 = nn.Conv2d(hidden_dim, hidden_dim, 3, padding=1)
        self.conv5 = nn.Conv2d(hidden_dim, 1, 3, padding=1)

    def forward(self, x, t):
        # Get time embedding and reshape for broadcasting
        t_emb = self.time_embed(t)  # [B, hidden_dim]
        t_emb = t_emb.view(-1, t_emb.shape[1], 1, 1)  # [B, hidden_dim, 1, 1]

        # Forward pass
        h = F.relu(self.conv1(x))
        h = h + t_emb  # Add time embedding
        h = F.relu(self.conv2(h))
        h = F.relu(self.conv3(h))
        h = F.relu(self.conv4(h))
        h = self.conv5(h)
        return h


model = SimpleNoisePredictor(T=T)
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

### Step 4: Implement the DDPM Training Step

In [ ]:
#@title 🎧 Code Walkthrough: Ddpm Train Step
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_09_ddpm_train_step.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def ddpm_train_step(model, x_0, optimizer):
    """One training step: sample t, add noise, predict noise, compute MSE."""
    batch_size = x_0.shape[0]

    # 1. Sample random timesteps
    t = torch.randint(0, T, (batch_size,))

    # 2. Sample random noise
    noise = torch.randn_like(x_0)

    # 3. Create noisy images
    x_t, _ = forward_diffusion(x_0, t, noise)

    # 4. Predict noise
    predicted_noise = model(x_t, t)

    # 5. Compute MSE loss
    loss = F.mse_loss(predicted_noise, noise)

    # 6. Optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item()

In [ ]:
#@title 🎧 Transition: Checkpoint Training
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_10_checkpoint_training.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


**Checkpoint:** This is the entire DDPM training procedure. Sample noise, add it to images, predict it, minimize MSE. Beautifully simple.

## 5. Your Turn

### TODO 1: Train and Monitor the Loss

In [ ]:
#@title 🎧 Before You Start: Todo Train Loss
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_11_todo_1_train_loss.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def train_and_plot(model, dataloader, num_epochs=5, lr=1e-3):
    """
    TODO: Train the noise predictor and plot the loss curve.

    1. Create an Adam optimizer
    2. Loop over epochs and batches
    3. Call ddpm_train_step for each batch
    4. Record the loss every 50 steps
    5. Plot the loss curve
    6. Print the final loss

    Expected: Loss should decrease from ~1.0 to ~0.3-0.5
    """
    # HINT: The loss should steadily decrease, showing the model
    # is learning to predict noise better over time.

    pass  # YOUR CODE HERE

In [ ]:
#@title 🎧 Listen: Todo After
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_12_todo_1_after.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### TODO 2: Visualize Noise Predictions

In [ ]:
#@title 🎧 Before You Start: Todo Visualize Noise
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_13_todo_2_visualize_noise.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def visualize_noise_predictions(model, x_0, timesteps=[100, 300, 500, 700, 900]):
    """
    TODO: For a given clean image, show the noise prediction at different timesteps.

    For each timestep:
    1. Add noise to x_0 to get x_t (using forward_diffusion)
    2. Predict the noise using the model
    3. Show side-by-side: x_t, true noise, predicted noise, |error|

    This will reveal how well the model predicts noise at different levels.
    """
    # HINT: Use model.eval() and torch.no_grad() for inference

    pass  # YOUR CODE HERE

In [ ]:
#@title 🎧 Listen: Todo After
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_14_todo_2_after.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Code Walkthrough: Putting It All Together
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_15_putting_it_all_together.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 6. Putting It All Together

Let us train our simple model and see the noise prediction in action.

In [ ]:
#@title 🎧 What to Look For: Training Results Loss
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_16_training_results_loss.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Train for a few epochs
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
losses = []

model.train()
for epoch in range(3):
    epoch_losses = []
    for batch_idx, (images, _) in enumerate(dataloader):
        loss = ddpm_train_step(model, images, optimizer)
        epoch_losses.append(loss)
        if batch_idx % 200 == 0:
            print(f"Epoch {epoch+1}, Batch {batch_idx}, Loss: {loss:.4f}")
    avg_loss = np.mean(epoch_losses)
    losses.extend(epoch_losses)
    print(f"Epoch {epoch+1} average loss: {avg_loss:.4f}")

plt.figure(figsize=(10, 4))
plt.plot(losses[::10])
plt.xlabel('Training Steps (x10)')
plt.ylabel('MSE Loss')
plt.title('DDPM Training Loss')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
#@title 🎧 What to Look For: Visualize Noise Detailed
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_17_visualize_noise_detailed.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 7. Training and Results

In [ ]:
# Visualize the model's noise predictions
model.eval()
sample = dataset[0][0].unsqueeze(0)

fig, axes = plt.subplots(4, 5, figsize=(20, 16))
timesteps = [50, 200, 400, 600, 900]
row_labels = ['Noisy xₜ', 'True Noise ε', 'Predicted ε̂', '|Error|']

for col, t in enumerate(timesteps):
    t_tensor = torch.tensor([t])
    noise = torch.randn_like(sample)
    x_t, _ = forward_diffusion(sample, t_tensor, noise)

    with torch.no_grad():
        pred_noise = model(x_t, t_tensor)

    error = (noise - pred_noise).abs()

    axes[0][col].imshow(x_t.squeeze().clamp(-1, 1).detach().numpy(), cmap='gray')
    axes[0][col].set_title(f't={t}')
    axes[1][col].imshow(noise.squeeze().numpy(), cmap='gray')
    axes[2][col].imshow(pred_noise.squeeze().detach().numpy(), cmap='gray')
    axes[3][col].imshow(error.squeeze().detach().numpy(), cmap='hot')

for row, label in enumerate(row_labels):
    axes[row][0].set_ylabel(label, fontsize=12, rotation=0, labelpad=80, va='center')

for ax in axes.flat:
    ax.axis('off')

plt.suptitle('Noise Prediction at Different Timesteps', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 What to Look For: Final Output Reconstruction
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_18_final_output_reconstruction.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


**Checkpoint:** You should see that the predicted noise pattern roughly matches the true noise, especially at lower timesteps where there is more signal. The error map should show that the model is learning!

## 8. Final Output

In [ ]:
# Show the key insight: noise prediction enables mean estimation
model.eval()
sample = dataset[5][0].unsqueeze(0)

fig, axes = plt.subplots(2, 5, figsize=(20, 8))
timesteps = [100, 300, 500, 700, 900]

for col, t in enumerate(timesteps):
    t_tensor = torch.tensor([t])
    noise = torch.randn_like(sample)
    x_t, _ = forward_diffusion(sample, t_tensor, noise)

    with torch.no_grad():
        pred_noise = model(x_t, t_tensor)

    # Reconstruct x_0 from predicted noise
    sqrt_ab = torch.sqrt(alpha_bars[t])
    sqrt_1_ab = torch.sqrt(1 - alpha_bars[t])
    x_0_pred = (x_t - sqrt_1_ab * pred_noise) / sqrt_ab

    axes[0][col].imshow(x_t.squeeze().clamp(-1, 1).detach().numpy(), cmap='gray')
    axes[0][col].set_title(f'Noisy x_t (t={t})')
    axes[1][col].imshow(x_0_pred.squeeze().clamp(-1, 1).detach().numpy(), cmap='gray')
    axes[1][col].set_title(f'Predicted x₀')

for ax in axes.flat:
    ax.axis('off')

plt.suptitle('From Noise Prediction to Image Reconstruction', fontsize=16)
plt.tight_layout()
plt.show()
print("\nBy predicting the noise, we can estimate the original clean image from any noise level!")
print("This is the core insight of DDPM — training a generative model reduces to denoising.")

In [ ]:
#@title 🎧 Wrap-Up: Closing
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_19_closing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 9. Reflection and Next Steps

### Key Takeaways
1. The true reverse posterior $q(x_{t-1}|x_t, x_0)$ is a Gaussian with known mean and variance
2. The DDPM loss simplifies to predicting the added noise: $\|\epsilon - \epsilon_\theta(x_t, t)\|^2$
3. Noise prediction is equivalent to estimating the score function $\nabla \log p(x_t)$
4. A simple CNN can already learn basic noise prediction — U-Nets make it much better

### Reflection Questions
- Why does predicting noise work better than directly predicting $x_0$?
- At which timesteps do you think noise prediction is hardest? Why?
- How does the loss function relate to the variational lower bound (ELBO)?

### What is Next
In the next notebook, we will build a proper U-Net architecture with timestep embeddings and train a full DDPM that can actually generate convincing images from pure noise.